In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.layers import Lambda
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from keras import backend as K
import pywt
import numpy as np
from keras.engine.topology import Layer

def mkwtarray(image):
    channels = K.image_data_format()
    if channels is 'channels_first':
        axbase = 1
    else:
        axbase = 0
    (a,( b, c, d ))= pywt.dwt2(image, 'db1', axes=(axbase, axbase+1))
    ab = np.concatenate((a, b), axis=axbase)
    cd = np.concatenate((c, d), axis=axbase)
    abcd = np.concatenate((ab, cd), axis=axbase+1)
    return abcd

def myswt(image):
    channels = K.image_data_format()
    if channels is 'channels_first':
        axbase = 1
    else:
        axbase = 0
    res= pywt.swt2(image, 'db1', 1,axes=(axbase, axbase+1))[0][0]
    return res

# dimensions of our images.
img_width, img_height = 1536, 1536

train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 34810
nb_validation_samples = 344
epochs = 2
batch_size = 8

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
#model.add(MyLayer( input_shape=input_shape))
model.add(Conv2D(1, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(BatchNormalization())

model.add(Conv2D(2, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#add another layer
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')
checkpointer = ModelCheckpoint(filepath='/home/igor/nowt.hdf5', verbose=1, save_best_only=True)
history1 = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks = [checkpointer])


Found 34796 images belonging to 5 classes.
Found 330 images belonging to 5 classes.
Epoch 1/2
4351/4351 [==============================] - 18889s 4s/step - loss: 0.9214 - acc: 0.7341 - val_loss: 1.4828 - val_acc: 0.5917

Epoch 00001: val_loss improved from inf to 1.48277, saving model to /home/igor/nowt.hdf5
Epoch 2/2
4351/4351 [==============================] - 18754s 4s/step - loss: 0.8714 - acc: 0.7361 - val_loss: 1.3953 - val_acc: 0.5947

Epoch 00002: val_loss improved from 1.48277 to 1.39531, saving model to /home/igor/nowt.hdf5


In [3]:
history2 = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks = [checkpointer])

Epoch 1/2
4351/4351 [==============================] - 18110s 4s/step - loss: 0.8626 - acc: 0.7362 - val_loss: 1.3790 - val_acc: 0.6006

Epoch 00001: val_loss improved from 1.39531 to 1.37897, saving model to /home/igor/nowt.hdf5
Epoch 2/2
4351/4351 [==============================] - 17820s 4s/step - loss: 0.8617 - acc: 0.7361 - val_loss: 1.4344 - val_acc: 0.5858

Epoch 00002: val_loss did not improve from 1.37897


In [4]:
import os

In [5]:
!ls

pretrained_try.h5	  test		train		     Untitled1.ipynb
process.ipynb		  test.zip	trainLabels.csv      Untitled2.ipynb
retinaswt.h5		  test.zip.001	trainLabels.csv.zip  Untitled3.ipynb
retina_try.h5		  test.zip.002	train.zip	     Untitled4.ipynb
sample			  test.zip.003	train.zip.001	     Untitled.ipynb
sampleSubmission.csv.zip  test.zip.004	train.zip.002	     validation
sample.zip		  test.zip.005	train.zip.003
simplenetdeeperwww.py	  test.zip.006	train.zip.004
simplenetdeeperwww.py~	  test.zip.007	train.zip.005


In [6]:
os.listdir('test')

['5639_right.jpeg',
 '661_left.jpeg',
 '44041_left.jpeg',
 '14458_left.jpeg',
 '12808_right.jpeg',
 '3841_left.jpeg',
 '6143_right.jpeg',
 '24644_right.jpeg',
 '5730_left.jpeg',
 '25439_right.jpeg',
 '15229_right.jpeg',
 '36550_right.jpeg',
 '5068_left.jpeg',
 '38483_right.jpeg',
 '9429_left.jpeg',
 '35133_right.jpeg',
 '3200_right.jpeg',
 '23359_left.jpeg',
 '15046_right.jpeg',
 '32869_left.jpeg',
 '43928_right.jpeg',
 '12346_left.jpeg',
 '21990_right.jpeg',
 '34538_right.jpeg',
 '8279_left.jpeg',
 '29163_right.jpeg',
 '5830_left.jpeg',
 '34199_right.jpeg',
 '19449_right.jpeg',
 '19128_right.jpeg',
 '915_right.jpeg',
 '36194_right.jpeg',
 '7325_right.jpeg',
 '10878_left.jpeg',
 '6026_left.jpeg',
 '43000_left.jpeg',
 '19173_left.jpeg',
 '35148_right.jpeg',
 '409_right.jpeg',
 '31269_left.jpeg',
 '42739_left.jpeg',
 '31728_right.jpeg',
 '30532_right.jpeg',
 '38079_left.jpeg',
 '25313_right.jpeg',
 '33648_right.jpeg',
 '24529_right.jpeg',
 '37473_right.jpeg',
 '7956_left.jpeg',
 '32048_r

In [9]:
import skimage

In [8]:
import Pillow as pil

ModuleNotFoundError: No module named 'Pillow'

In [13]:
theim = skimage.io.imread('test/5639_right.jpeg')

In [11]:
import skimage.io

In [14]:
from skimage.transform import rescale, resize

In [19]:
theim2 = resize(theim, (1536, 1536))

/home/igor/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [20]:
theim3 = np.array(theim2)

In [21]:
model.predict(np.array([theim3]))

array([[0.7281351 , 0.06804642, 0.15403637, 0.02740168, 0.02238038]],
      dtype=float32)

In [22]:
np.argmax(_)

0

In [26]:
def fname_predict(fname, mm):
    fullname = 'test/' + fname
    theim = np.array(resize(skimage.io.imread(fullname), (1536, 1536)))
    return np.argmax(mm.predict(np.array([theim])))

In [27]:
fname_predict('5639_right.jpeg', model)

/home/igor/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


0

In [29]:
res = []
for i in os.listdir('test'):
    pp = fname_predict(i, model)
    print(i, pp)
    res.append((i, pp))

/home/igor/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


5639_right.jpeg 0
661_left.jpeg 0
44041_left.jpeg 0
14458_left.jpeg 0
12808_right.jpeg 0
3841_left.jpeg 0
6143_right.jpeg 0
24644_right.jpeg 0
5730_left.jpeg 0
25439_right.jpeg 0
15229_right.jpeg 0
36550_right.jpeg 0
5068_left.jpeg 0
38483_right.jpeg 0
9429_left.jpeg 0
35133_right.jpeg 0
3200_right.jpeg 0
23359_left.jpeg 0
15046_right.jpeg 0
32869_left.jpeg 0
43928_right.jpeg 0
12346_left.jpeg 0
21990_right.jpeg 0
34538_right.jpeg 0
8279_left.jpeg 0
29163_right.jpeg 0
5830_left.jpeg 0
34199_right.jpeg 0
19449_right.jpeg 0
19128_right.jpeg 0
915_right.jpeg 0
36194_right.jpeg 0
7325_right.jpeg 0
10878_left.jpeg 0
6026_left.jpeg 0
43000_left.jpeg 0
19173_left.jpeg 0
35148_right.jpeg 0
409_right.jpeg 0
31269_left.jpeg 0
42739_left.jpeg 0
31728_right.jpeg 0
30532_right.jpeg 0
38079_left.jpeg 0
25313_right.jpeg 0
33648_right.jpeg 0
24529_right.jpeg 0
37473_right.jpeg 0
7956_left.jpeg 0
32048_right.jpeg 0
27652_left.jpeg 0
19007_right.jpeg 0
37789_left.jpeg 0
40414_left.jpeg 0
15543_left.jpeg

9089_right.jpeg 0
15860_left.jpeg 0
27638_right.jpeg 0
26053_left.jpeg 0
20265_left.jpeg 0
38872_left.jpeg 0
17954_right.jpeg 0
15350_left.jpeg 0
17980_left.jpeg 0
42169_right.jpeg 0
5802_right.jpeg 0
7610_left.jpeg 0
20846_left.jpeg 0
41093_left.jpeg 0
25164_left.jpeg 0
43497_left.jpeg 0
42046_left.jpeg 0
2127_left.jpeg 0
7203_left.jpeg 0
30951_left.jpeg 0
38177_left.jpeg 0
5787_right.jpeg 0
30890_left.jpeg 0
11339_right.jpeg 0
15266_right.jpeg 0
19459_left.jpeg 0
10595_right.jpeg 0
35728_left.jpeg 0
17336_left.jpeg 0
38086_left.jpeg 0
42822_left.jpeg 0
39116_right.jpeg 0
36887_left.jpeg 0
37445_right.jpeg 0
11309_left.jpeg 0
25795_left.jpeg 0
1341_left.jpeg 0
28877_right.jpeg 0
2100_right.jpeg 0
2277_right.jpeg 0
17828_right.jpeg 0
33048_right.jpeg 0
26244_left.jpeg 0
17978_right.jpeg 0
13428_left.jpeg 0
33968_left.jpeg 0
24069_left.jpeg 0
22111_left.jpeg 0
10316_left.jpeg 0
8992_right.jpeg 0
22243_left.jpeg 0
21029_left.jpeg 0
3775_left.jpeg 0
38483_left.jpeg 0
23564_left.jpeg 0
333

3013_right.jpeg 0
16208_left.jpeg 0
30801_right.jpeg 0
1925_left.jpeg 0
14396_left.jpeg 0
33136_left.jpeg 0
5719_right.jpeg 0
5704_left.jpeg 0
9866_right.jpeg 0
20658_left.jpeg 0
14557_right.jpeg 0
18216_right.jpeg 0
27882_left.jpeg 0
8322_right.jpeg 0
29157_left.jpeg 0
29125_right.jpeg 0
1860_right.jpeg 0
1031_right.jpeg 0
7720_right.jpeg 0
4492_right.jpeg 0
20818_left.jpeg 0
39662_right.jpeg 0
14578_right.jpeg 0
37373_left.jpeg 0
30162_left.jpeg 0
35534_left.jpeg 0
25655_right.jpeg 0
3236_right.jpeg 0
7146_right.jpeg 0
39531_left.jpeg 0
42750_right.jpeg 0
213_left.jpeg 0
30450_left.jpeg 0
30086_left.jpeg 0
33373_left.jpeg 0
28739_left.jpeg 0
34723_left.jpeg 0
14576_left.jpeg 0
3191_left.jpeg 0
3878_right.jpeg 0
15316_right.jpeg 0
13411_right.jpeg 0
31937_left.jpeg 0
23638_left.jpeg 0
4663_right.jpeg 0
9461_left.jpeg 0
37706_right.jpeg 0
40661_right.jpeg 0
37140_right.jpeg 0
32384_right.jpeg 0
39586_right.jpeg 0
32294_right.jpeg 0
21780_left.jpeg 0
26791_left.jpeg 0
7809_left.jpeg 0
7

824_left.jpeg 0
34790_right.jpeg 0
34017_right.jpeg 0
39093_left.jpeg 0
30543_left.jpeg 0
10959_right.jpeg 0
39770_right.jpeg 0
22313_left.jpeg 0
3164_left.jpeg 0
5575_left.jpeg 0
20010_left.jpeg 0
13183_left.jpeg 0
3517_left.jpeg 0
41817_left.jpeg 0
16526_right.jpeg 0
35752_right.jpeg 0
12833_left.jpeg 0
39277_right.jpeg 0
31410_left.jpeg 0
37902_right.jpeg 0
3445_left.jpeg 0
37862_left.jpeg 0
16328_left.jpeg 0
18360_right.jpeg 0
20387_right.jpeg 0
9_right.jpeg 0
11454_left.jpeg 0
38586_left.jpeg 0
26400_left.jpeg 0
39450_right.jpeg 0
35709_right.jpeg 0
43200_right.jpeg 0
8240_left.jpeg 0
23760_right.jpeg 0
41345_left.jpeg 0
41329_left.jpeg 0
12097_right.jpeg 0
4355_left.jpeg 0
30811_right.jpeg 0
468_left.jpeg 0
25850_left.jpeg 0
6419_left.jpeg 0
4745_right.jpeg 0
36394_left.jpeg 0
21945_left.jpeg 0
28034_right.jpeg 0
18393_right.jpeg 0
37239_right.jpeg 0
41720_right.jpeg 0
17711_right.jpeg 0
1804_right.jpeg 0
43401_left.jpeg 0
6279_right.jpeg 0
19706_left.jpeg 0
41498_right.jpeg 0
94

33004_left.jpeg 0
9557_left.jpeg 0
40173_left.jpeg 0
16070_left.jpeg 0
35213_right.jpeg 0
44304_right.jpeg 0
22892_left.jpeg 0
40710_left.jpeg 0
11918_right.jpeg 0
40846_left.jpeg 0
24970_right.jpeg 0
14319_right.jpeg 0
18484_left.jpeg 0
41341_right.jpeg 0
12526_left.jpeg 0
42240_left.jpeg 0
26500_left.jpeg 0
9062_right.jpeg 0
10185_right.jpeg 0
42366_left.jpeg 0
21136_left.jpeg 0
41933_left.jpeg 0
5157_right.jpeg 0
42102_right.jpeg 0
11017_right.jpeg 0
8139_left.jpeg 0
34314_right.jpeg 0
22009_right.jpeg 0
31977_right.jpeg 0
36106_right.jpeg 0
25384_right.jpeg 0
37787_left.jpeg 0
39515_left.jpeg 0
15568_right.jpeg 0
8626_right.jpeg 0
23070_left.jpeg 0
29745_right.jpeg 0
38562_right.jpeg 0
8665_left.jpeg 0
22197_left.jpeg 0
26504_left.jpeg 0
38669_right.jpeg 0
2936_right.jpeg 0
34247_right.jpeg 0
2348_right.jpeg 0
7386_left.jpeg 0
4661_right.jpeg 0
8639_right.jpeg 0
36949_right.jpeg 0
37512_right.jpeg 0
37044_left.jpeg 0
35157_left.jpeg 0
34544_left.jpeg 0
38654_left.jpeg 0
37815_left.

12816_right.jpeg 0
6395_left.jpeg 0
30023_right.jpeg 0
43671_right.jpeg 0
23639_right.jpeg 0
35141_left.jpeg 0
4597_right.jpeg 0
9070_right.jpeg 0
589_right.jpeg 0
23582_right.jpeg 0
7687_right.jpeg 0
4024_right.jpeg 0
17285_left.jpeg 0
2777_left.jpeg 0
1518_right.jpeg 0
11059_right.jpeg 0
37558_right.jpeg 0
4240_right.jpeg 0
7261_right.jpeg 0
33086_right.jpeg 0
25350_right.jpeg 0
27787_left.jpeg 0
6932_left.jpeg 0
12407_left.jpeg 0
27748_left.jpeg 0
14277_left.jpeg 0
9738_right.jpeg 0
22794_left.jpeg 0
34909_left.jpeg 0
29687_right.jpeg 0
15697_right.jpeg 0
34009_left.jpeg 0
9936_right.jpeg 0
31404_left.jpeg 0
36615_right.jpeg 0
44109_right.jpeg 0
12047_left.jpeg 0
26826_right.jpeg 0
1815_left.jpeg 0
38614_left.jpeg 0
31611_right.jpeg 0
12640_right.jpeg 0
5445_left.jpeg 0
2856_right.jpeg 0
32161_right.jpeg 0
6946_left.jpeg 0
7266_right.jpeg 0
41783_right.jpeg 0
33116_left.jpeg 0
3576_left.jpeg 0
8010_left.jpeg 0
35786_right.jpeg 0
22705_right.jpeg 0
41439_left.jpeg 0
6564_right.jpeg 0

KeyboardInterrupt: 

In [30]:
res

[('5639_right.jpeg', 0),
 ('661_left.jpeg', 0),
 ('44041_left.jpeg', 0),
 ('14458_left.jpeg', 0),
 ('12808_right.jpeg', 0),
 ('3841_left.jpeg', 0),
 ('6143_right.jpeg', 0),
 ('24644_right.jpeg', 0),
 ('5730_left.jpeg', 0),
 ('25439_right.jpeg', 0),
 ('15229_right.jpeg', 0),
 ('36550_right.jpeg', 0),
 ('5068_left.jpeg', 0),
 ('38483_right.jpeg', 0),
 ('9429_left.jpeg', 0),
 ('35133_right.jpeg', 0),
 ('3200_right.jpeg', 0),
 ('23359_left.jpeg', 0),
 ('15046_right.jpeg', 0),
 ('32869_left.jpeg', 0),
 ('43928_right.jpeg', 0),
 ('12346_left.jpeg', 0),
 ('21990_right.jpeg', 0),
 ('34538_right.jpeg', 0),
 ('8279_left.jpeg', 0),
 ('29163_right.jpeg', 0),
 ('5830_left.jpeg', 0),
 ('34199_right.jpeg', 0),
 ('19449_right.jpeg', 0),
 ('19128_right.jpeg', 0),
 ('915_right.jpeg', 0),
 ('36194_right.jpeg', 0),
 ('7325_right.jpeg', 0),
 ('10878_left.jpeg', 0),
 ('6026_left.jpeg', 0),
 ('43000_left.jpeg', 0),
 ('19173_left.jpeg', 0),
 ('35148_right.jpeg', 0),
 ('409_right.jpeg', 0),
 ('31269_left.jpeg'

In [31]:
def fname_probs(fname, mm):
    fullname = 'test/' + fname
    theim = np.array(resize(skimage.io.imread(fullname), (1536, 1536)))
    return mm.predict(np.array([theim]))

In [33]:
res2 = {}
for i in os.listdir('test'):
    pp = fname_probs(i, model)
    print(i, pp)
    res2[i] = pp

/home/igor/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


5639_right.jpeg [[0.7281351  0.06804642 0.15403637 0.02740168 0.02238038]]
661_left.jpeg [[0.7154656  0.06834404 0.16346052 0.02828505 0.02444483]]
44041_left.jpeg [[0.7154656  0.06834404 0.16346052 0.02828505 0.02444483]]
14458_left.jpeg [[0.716721   0.0684286  0.16189362 0.02830662 0.02465014]]
12808_right.jpeg [[0.74946535 0.07006431 0.1352733  0.0242086  0.02098855]]
3841_left.jpeg [[0.7154656  0.06834404 0.16346052 0.02828505 0.02444483]]
6143_right.jpeg [[0.7154656  0.06834404 0.16346052 0.02828505 0.02444483]]
24644_right.jpeg [[0.7154656  0.06834404 0.16346052 0.02828505 0.02444483]]
5730_left.jpeg [[0.7154656  0.06834404 0.16346052 0.02828505 0.02444483]]
25439_right.jpeg [[0.7154656  0.06834404 0.16346052 0.02828505 0.02444483]]
15229_right.jpeg [[0.7219609  0.06818225 0.15752947 0.02808589 0.02424152]]
36550_right.jpeg [[0.7154656  0.06834404 0.16346052 0.02828505 0.02444483]]
5068_left.jpeg [[0.7154656  0.06834404 0.16346052 0.02828505 0.02444483]]
38483_right.jpeg [[0.7154

KeyboardInterrupt: 

In [34]:
import pywt

In [35]:
pywt.wavedec2(theim3[0, 'db1', level=4)

ValueError: Level value of 4 is too high.  Maximum allowed is 1.

In [36]:
theim3[0]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [37]:
theim3[0].shape

(1536, 3)

In [38]:
theim3[:,:,0].shape

(1536, 1536)

In [43]:
tt =pywt.wavedec2(theim3[:, :, 0], 'db1', level=4)

In [42]:
len(_)

5

In [44]:
tt[0]

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00813874],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [45]:
tt[0].shape

(96, 96)

In [46]:
np.linalg.norm(tt[0])

440.7976084929614

In [47]:
np.linalg.norm(tt[1])

25.81954663295863

In [48]:
np.linalg.norm(tt[2])

18.760939799405854

In [50]:
np.linalg.norm(tt[3])

13.869553106228313

In [51]:
np.linalg.norm(tt[4])

11.562946761977296

In [52]:
def get_norms(img, lev=10, res=3072, num_chan=3):
    res = []
    for i in range(num_chan):
        thedwt = pywt.wavedec2(img[:, :, i], 'db1', level=lev)
        for i in range(lev+1):
            curlev = thedwt[i]
            res.append(np.linalg.norm(curlev))
    return res
        
    

In [53]:
foo = get_norms(theim3, lev=9)

In [54]:
foo

[380.8022308465839,
 166.66656421679846,
 110.74760844700805,
 72.75884335131099,
 52.89550090629331,
 34.02400518556022,
 25.81954663295863,
 18.760939799405854,
 13.869553106228313,
 11.562946761977296,
 280.936527331411,
 121.3137310061969,
 81.64751643787014,
 53.11327974752373,
 37.782504350231704,
 26.624246236529785,
 20.757068590762344,
 15.071322321688404,
 11.240380233907972,
 10.218062076732847,
 230.5271060169122,
 101.59214933143888,
 68.05700417535179,
 44.74260065882538,
 31.33067807181879,
 20.869719470004107,
 15.724839627727698,
 11.792634668386325,
 9.782333730400863,
 9.954531468309689]

In [55]:
def fname_norms(fname, cl, res=(3072,3072), lev=10):
    fullname = 'train/'  + cl + '/' + fname
    theim = np.array(resize(skimage.io.imread(fullname), res))
    return get_norms(theim, lev=10)

In [56]:
os.listdir('train/0')

['25156_right.jpeg',
 '17693_right.jpeg',
 '15340_right.jpeg',
 '37038_right.jpeg',
 '18226_left.jpeg',
 '2933_right.jpeg',
 '27556_left.jpeg',
 '6578_left.jpeg',
 '7701_left.jpeg',
 '25631_right.jpeg',
 '25148_left.jpeg',
 '3023_right.jpeg',
 '23903_right.jpeg',
 '6237_left.jpeg',
 '42998_right.jpeg',
 '2516_left.jpeg',
 '7438_right.jpeg',
 '1506_left.jpeg',
 '6141_left.jpeg',
 '26756_left.jpeg',
 '36178_left.jpeg',
 '17768_left.jpeg',
 '13673_left.jpeg',
 '40650_right.jpeg',
 '2413_left.jpeg',
 '37146_right.jpeg',
 '5720_right.jpeg',
 '329_left.jpeg',
 '40339_right.jpeg',
 '9789_right.jpeg',
 '13937_left.jpeg',
 '13251_right.jpeg',
 '5976_left.jpeg',
 '8227_left.jpeg',
 '21896_left.jpeg',
 '34606_left.jpeg',
 '9767_right.jpeg',
 '35209_left.jpeg',
 '31834_right.jpeg',
 '41922_right.jpeg',
 '8433_right.jpeg',
 '6678_right.jpeg',
 '43758_left.jpeg',
 '10748_left.jpeg',
 '19890_left.jpeg',
 '24095_left.jpeg',
 '23217_right.jpeg',
 '36848_right.jpeg',
 '2491_left.jpeg',
 '26069_left.jpeg

In [57]:
foo = fname_norms('25156_right.jpeg', '0')

/home/igor/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [58]:
foo

[580.1721005736505,
 278.2736463728449,
 209.11779668428255,
 150.0489865993557,
 86.73169679370405,
 63.78868072612621,
 42.37970649130608,
 31.074839312308814,
 21.34428152954474,
 13.378553951935489,
 7.900789920663695,
 370.1797004371112,
 180.69249228013604,
 143.23367860718704,
 111.44633391040227,
 69.47855500707512,
 55.05330103240304,
 38.12357978671279,
 28.48445477864051,
 19.870776770986286,
 12.59085031448737,
 7.4220435540298695,
 160.30969181330659,
 80.8249510839777,
 67.3733623127993,
 60.02106589986531,
 38.17412105834207,
 30.925168816939063,
 21.28244653749812,
 17.599266274325934,
 14.585797247798132,
 10.564962462423761,
 6.714242247528091]

In [61]:
rr0 = []
for i in os.listdir('train/0'):
    print( i)
    ww = fname_norms(i, '0')
    rr0.append(ww)
df0 = pd.DataFrame(np.array(rr))

25156_right.jpeg


/home/igor/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


17693_right.jpeg
15340_right.jpeg
37038_right.jpeg
18226_left.jpeg
2933_right.jpeg
27556_left.jpeg
6578_left.jpeg
7701_left.jpeg
25631_right.jpeg
25148_left.jpeg
3023_right.jpeg
23903_right.jpeg
6237_left.jpeg
42998_right.jpeg
2516_left.jpeg
7438_right.jpeg
1506_left.jpeg
6141_left.jpeg
26756_left.jpeg
36178_left.jpeg
17768_left.jpeg
13673_left.jpeg
40650_right.jpeg
2413_left.jpeg
37146_right.jpeg
5720_right.jpeg
329_left.jpeg
40339_right.jpeg
9789_right.jpeg
13937_left.jpeg
13251_right.jpeg
5976_left.jpeg
8227_left.jpeg
21896_left.jpeg
34606_left.jpeg
9767_right.jpeg
35209_left.jpeg
31834_right.jpeg
41922_right.jpeg
8433_right.jpeg
6678_right.jpeg
43758_left.jpeg
10748_left.jpeg
19890_left.jpeg
24095_left.jpeg
23217_right.jpeg
36848_right.jpeg
2491_left.jpeg
26069_left.jpeg
25507_left.jpeg
35116_left.jpeg
18686_right.jpeg
13742_right.jpeg
28006_right.jpeg
19301_right.jpeg
19257_left.jpeg
12478_left.jpeg
41710_left.jpeg
2655_left.jpeg
34267_right.jpeg
29385_left.jpeg
7991_left.jpeg
418

42149_right.jpeg
8654_left.jpeg
23955_right.jpeg
40650_left.jpeg
25992_left.jpeg
41349_left.jpeg
40593_left.jpeg
19575_left.jpeg
39549_left.jpeg
38838_right.jpeg
25796_right.jpeg
13952_right.jpeg
15630_right.jpeg
38959_left.jpeg
31119_left.jpeg
27542_right.jpeg
33535_right.jpeg
8927_right.jpeg
27037_left.jpeg
12546_right.jpeg
32746_right.jpeg
17354_left.jpeg
17311_right.jpeg
2144_right.jpeg
39914_right.jpeg
8762_right.jpeg
20004_right.jpeg
40427_left.jpeg
30602_right.jpeg
29024_right.jpeg
35797_right.jpeg
23844_right.jpeg
28181_left.jpeg
35011_right.jpeg
25217_left.jpeg
20344_right.jpeg
33717_right.jpeg
18035_left.jpeg
19108_left.jpeg
7210_right.jpeg
24224_left.jpeg
38337_right.jpeg
11115_right.jpeg
27917_right.jpeg
35838_left.jpeg
15652_left.jpeg
24410_right.jpeg
42458_right.jpeg
35616_right.jpeg
5633_left.jpeg
11117_right.jpeg
11956_right.jpeg
16499_right.jpeg
22878_right.jpeg
35816_left.jpeg
41873_left.jpeg
4391_left.jpeg
4842_right.jpeg
1775_left.jpeg
27512_left.jpeg
29727_left.jpe

26490_right.jpeg
40299_right.jpeg
14697_left.jpeg
24665_left.jpeg
20194_left.jpeg
15755_left.jpeg
6970_right.jpeg
28906_right.jpeg
20573_right.jpeg
11340_left.jpeg
35402_left.jpeg
42488_right.jpeg
17143_left.jpeg
7568_left.jpeg
40043_left.jpeg
22068_left.jpeg
20782_left.jpeg
21344_right.jpeg
41835_right.jpeg
27742_left.jpeg
33970_right.jpeg
41645_right.jpeg
26651_left.jpeg
34496_right.jpeg
1799_left.jpeg
27664_left.jpeg
7736_right.jpeg
33791_right.jpeg
16547_right.jpeg
39956_left.jpeg
21368_left.jpeg
28019_right.jpeg
7204_right.jpeg
21720_right.jpeg
39574_right.jpeg
35650_left.jpeg
33368_right.jpeg
36537_right.jpeg
36585_left.jpeg
3406_left.jpeg
25079_left.jpeg
36526_left.jpeg
9133_left.jpeg
20208_left.jpeg
21699_right.jpeg
33303_left.jpeg
26184_right.jpeg
15453_right.jpeg
26007_right.jpeg
23601_right.jpeg
10594_left.jpeg
10622_left.jpeg
29412_right.jpeg
35459_left.jpeg
13473_left.jpeg
39315_left.jpeg
10429_left.jpeg
9513_left.jpeg
15602_left.jpeg
32470_right.jpeg
36624_right.jpeg
4145

...

23810_left.jpeg
19344_left.jpeg
36325_left.jpeg
6889_left.jpeg
40153_right.jpeg
9804_right.jpeg
25810_left.jpeg
29268_left.jpeg
39881_right.jpeg
43561_left.jpeg
39336_left.jpeg
14762_right.jpeg
8605_right.jpeg
3284_left.jpeg
31740_left.jpeg
12327_left.jpeg
1240_right.jpeg
41922_left.jpeg
19064_left.jpeg
43631_right.jpeg
37569_left.jpeg
36179_right.jpeg
39388_left.jpeg
36100_right.jpeg
3655_right.jpeg
30748_right.jpeg
41771_left.jpeg
32033_right.jpeg
1228_left.jpeg
25402_left.jpeg
23628_right.jpeg
31150_left.jpeg
16749_right.jpeg
40556_left.jpeg
20648_left.jpeg
9042_left.jpeg
17201_left.jpeg
1609_right.jpeg
26700_right.jpeg
11626_right.jpeg
8191_right.jpeg
9370_left.jpeg
7311_left.jpeg
27759_left.jpeg
16766_right.jpeg
38866_left.jpeg
41617_left.jpeg
36410_left.jpeg
33000_right.jpeg
1889_right.jpeg
18308_right.jpeg
11056_right.jpeg
37069_left.jpeg
35905_left.jpeg
31850_left.jpeg
28385_left.jpeg
15810_right.jpeg
17495_left.jpeg
37204_right.jpeg
9540_left.jpeg
20535_left.jpeg
26396_right.j

2874_left.jpeg
39671_right.jpeg
41006_right.jpeg
14036_right.jpeg
3700_left.jpeg
39930_right.jpeg
33124_right.jpeg
42382_left.jpeg
446_left.jpeg
42876_right.jpeg
625_left.jpeg
33348_left.jpeg
24155_left.jpeg
35987_right.jpeg
37028_right.jpeg
8835_left.jpeg
11476_right.jpeg
24416_left.jpeg
18056_right.jpeg
42004_left.jpeg
12700_left.jpeg
15848_left.jpeg
26955_right.jpeg
24863_left.jpeg
24878_left.jpeg
37450_left.jpeg
29512_right.jpeg
17357_right.jpeg
24009_left.jpeg
29939_left.jpeg
23002_left.jpeg
24592_left.jpeg
31477_right.jpeg
14877_right.jpeg
38799_right.jpeg
25685_left.jpeg
19458_left.jpeg
12349_left.jpeg
38011_right.jpeg
39878_left.jpeg
14258_right.jpeg
42317_right.jpeg
5782_left.jpeg
25573_right.jpeg
27852_right.jpeg
31769_right.jpeg
24193_right.jpeg
22466_left.jpeg
22201_right.jpeg
27472_right.jpeg
12876_right.jpeg
32405_left.jpeg
20775_left.jpeg
18315_right.jpeg
42625_left.jpeg
28018_left.jpeg
39679_left.jpeg
23535_left.jpeg
31104_left.jpeg
25829_left.jpeg
28204_left.jpeg
15638

16343_left.jpeg
3513_left.jpeg
23887_left.jpeg
42935_left.jpeg
26921_left.jpeg
12649_left.jpeg
42864_left.jpeg
10583_right.jpeg
39333_left.jpeg
41823_right.jpeg
13150_left.jpeg
9555_left.jpeg
24731_left.jpeg
30519_left.jpeg
2029_left.jpeg
3355_left.jpeg
9913_left.jpeg
22730_left.jpeg
37078_left.jpeg
18548_right.jpeg
6782_left.jpeg
36021_left.jpeg
38367_right.jpeg
36755_left.jpeg
20661_left.jpeg
40847_left.jpeg
21135_right.jpeg
30112_left.jpeg
17361_right.jpeg
39595_right.jpeg
41258_right.jpeg
23783_right.jpeg
40726_left.jpeg
12618_right.jpeg
38038_right.jpeg
20933_left.jpeg
7359_left.jpeg
29430_left.jpeg
3170_left.jpeg
762_left.jpeg
2721_left.jpeg
33546_right.jpeg
14716_right.jpeg
21438_right.jpeg
19854_right.jpeg
13325_right.jpeg
21110_left.jpeg
2842_left.jpeg
5590_right.jpeg
24941_right.jpeg
6273_left.jpeg
37617_left.jpeg
38258_right.jpeg
5752_left.jpeg
27816_left.jpeg
15516_right.jpeg
18870_left.jpeg
11392_right.jpeg
5083_left.jpeg
22822_right.jpeg
29588_left.jpeg
10658_left.jpeg
20

36465_right.jpeg
29921_right.jpeg
12597_right.jpeg
814_left.jpeg
2194_right.jpeg
20828_left.jpeg
23941_left.jpeg
1374_left.jpeg
33170_right.jpeg
16635_left.jpeg
17399_left.jpeg
37463_left.jpeg
29398_right.jpeg
15101_right.jpeg
9940_left.jpeg
42308_left.jpeg
5949_right.jpeg
26518_left.jpeg
2054_right.jpeg
20963_right.jpeg
11207_right.jpeg
599_right.jpeg
15973_left.jpeg
22519_right.jpeg
6705_right.jpeg
32598_right.jpeg
43259_left.jpeg
35794_left.jpeg
6920_right.jpeg
14672_left.jpeg
36471_left.jpeg
10962_left.jpeg
37041_right.jpeg
36422_right.jpeg
28211_right.jpeg
1586_right.jpeg
32031_left.jpeg
25069_right.jpeg
39716_right.jpeg
39604_left.jpeg
26375_right.jpeg
7613_right.jpeg
21881_right.jpeg
27009_right.jpeg
25713_left.jpeg
4323_right.jpeg
42157_right.jpeg
12474_left.jpeg
16533_left.jpeg
18069_right.jpeg
9971_right.jpeg
15567_left.jpeg
12146_left.jpeg
30754_left.jpeg
26421_left.jpeg
5186_left.jpeg
30749_right.jpeg
16693_right.jpeg
42804_left.jpeg
24782_right.jpeg
12633_right.jpeg
26973_

NameError: name 'pd' is not defined

In [ ]:
df0.to_csv('df0.csv')